In [1]:
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#path = "../../../Google Drive/Data_science/NYU/Machine Learning/ML Project (Collisions)/" #Joe
path = "../../../../Google Drive/ML Project (Collisions)/" # Joyce
# path = "" # Lucas

In [3]:
df = pd.read_csv(path + "NYPD_Motor_Vehicle_Collisions.csv", parse_dates=[['DATE', 'TIME']], infer_datetime_format=True)

In [4]:
new_df = pd.DataFrame()

In [5]:
new_df['day_of_week'] = df['DATE_TIME'].dt.dayofweek
new_df['hour_of_day'] = df['DATE_TIME'].dt.hour
new_df['year'] = df['DATE_TIME'].dt.year
new_df['month'] = df['DATE_TIME'].dt.month
new_df['day'] = df['DATE_TIME'].dt.day
new_df['time_of_day'] = df['DATE_TIME'].dt.hour*60 + df['DATE_TIME'].dt.minute

In [6]:
renamed_columns = [x.lower().replace(' ', '_') for x in df.columns.values]
    
df.columns = renamed_columns

In [7]:
new_df['num_vehicles'] = df.vehicle_type_code_1.notnull().astype(int)+df.vehicle_type_code_2.notnull().astype(int)\
                +df.vehicle_type_code_3.notnull().astype(int)+df.vehicle_type_code_4.notnull().astype(int)\
                +df.vehicle_type_code_5.notnull().astype(int)
        
df['num_vehicles'] = df.vehicle_type_code_1.notnull().astype(int)+df.vehicle_type_code_2.notnull().astype(int)\
                +df.vehicle_type_code_3.notnull().astype(int)+df.vehicle_type_code_4.notnull().astype(int)\
                +df.vehicle_type_code_5.notnull().astype(int)

In [8]:
remap_vehicle = {'TAXI': 'SMALL COM VEH(4 TIRES)',
                'BUS': 'LARGE VEHICLE',
                'SCOOTER': 'MOTORCYCLE',
                'LIVERY VEHICLE': 'SMALL COM VEH(4 TIRES)',
                'VAN': 'MEDIUM VEHICLE',
                'PEDICAB': 'BICYCLE',
                'PICK-UP TRUCK': 'MEDIUM VEHICLE',
                'SPORT UTILITY / STATION WAGON': 'MEDIUM VEHICLE',
                'LARGE COM VEH(6 OR MORE TIRES)': 'LARGE VEHICLE',
                'FIRE TRUCK': 'LARGE VEHICLE',
                }

In [9]:
for column in ['vehicle_type_code_1', 'vehicle_type_code_2',
       'vehicle_type_code_3', 'vehicle_type_code_4', 'vehicle_type_code_5']:
    new_df[column] = df[column].replace(remap_vehicle)

In [10]:
new_df['vehicle_type_code_1'].unique()

array(['MEDIUM VEHICLE', 'PASSENGER VEHICLE', nan,
       'SMALL COM VEH(4 TIRES)', 'BICYCLE', 'OTHER', 'LARGE VEHICLE',
       'MOTORCYCLE', 'AMBULANCE', 'UNKNOWN'], dtype=object)

In [11]:
def all_same(vehlist, num_vehicles):
    if num_vehicles < 2:
        return False
    else:
        return all(x == vehlist[0] for x in vehlist[:num_vehicles])

In [12]:
new_df['all_vehicles_same_type'] = new_df.apply(lambda row: all_same([row['vehicle_type_code_1'],\
                                                              row['vehicle_type_code_2'],\
                                                              row['vehicle_type_code_3'],\
                                                              row['vehicle_type_code_4'],\
                                                              row['vehicle_type_code_5']], row['num_vehicles']), axis=1)

In [13]:
for column in new_df.vehicle_type_code_1.unique():
    
    col1 = (new_df['vehicle_type_code_1'] == column).astype(int)
    col2 = (new_df['vehicle_type_code_2'] == column).astype(int)
    col3 = (new_df['vehicle_type_code_3'] == column).astype(int)
    col4 = (new_df['vehicle_type_code_4'] == column).astype(int)
    col5 = (new_df['vehicle_type_code_5'] == column).astype(int)
    
    new_df['num '+str(column)] = col1+col2+col3+col4+col5

In [14]:
new_df['injured'] = (df.number_of_persons_injured+df.number_of_pedestrians_injured+\
                df.number_of_cyclist_injured+df.number_of_motorist_injured)>0
new_df['killed'] = (df.number_of_persons_killed+df.number_of_pedestrians_killed+\
                df.number_of_cyclist_killed+df.number_of_motorist_killed)>0
new_df['injured_or_killed'] = new_df.injured | new_df.killed

In [15]:
renamed_columns = [x.lower().replace(' ', '_') for x in new_df.columns.values]
    
new_df.columns = renamed_columns

In [16]:
total = np.sum((new_df['num_bicycle'] > 0))
percent = np.sum((new_df['num_bicycle'] > 0) & (new_df.injured_or_killed > 0))/total

print ("Num bicycles > 0: %.2f percent, %s counts" % (percent*100, total))

Num bicycles > 0: 78.19 percent, 25423 counts


In [17]:
total = np.sum((new_df['num_vehicles'] == 1) & (new_df['num_bicycle'] == 0))
percent = np.sum((new_df['num_vehicles'] == 1) & (new_df.injured_or_killed > 0) & (new_df['num_bicycle'] == 0))/total

print ("Num vehicles = 1: %.2f percent, %s counts" % (percent*100, total))


Num vehicles = 1: 45.43 percent, 130334 counts


In [18]:
total = np.sum((new_df['num_vehicles'] == 0))
percent = np.sum((new_df['num_vehicles'] == 0) & (new_df.injured_or_killed > 0))/total

print ("Num vehicles = 0: %.2f percent, %s counts" % (percent*100, total))

Num vehicles = 0: 28.31 percent, 5006 counts


In [19]:
total = np.sum((new_df['num_vehicles'] > 1) & (new_df['num_bicycle'] == 0))
percent = np.sum((new_df['num_vehicles'] > 1) & (new_df.injured_or_killed > 0) & (new_df['num_bicycle'] == 0))/total

print ("Num vehicles > 1: %.2f percent, %s counts" % (percent*100, total))

Num vehicles > 1: 12.72 percent, 833736 counts


In [20]:
new_df['UNIQUE KEY'] = df['unique_key']
new_df.set_index('UNIQUE KEY', inplace=True)

In [21]:
new_df = new_df.drop(['vehicle_type_code_1', 'vehicle_type_code_2', 'vehicle_type_code_3', 
        'vehicle_type_code_4', 'vehicle_type_code_5'], axis=1)

In [22]:
with open(path+"data_for_joining/self.pkl", 'wb') as outfile:
    pickle.dump(new_df, outfile)